In [ ]:
### TEMP 



# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_colwidth', None)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from matplotlib import pyplot as plt
import seaborn as sns

# from plotly.offline import init_notebook_mode
# init_notebook_mode(connected=True)
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go

import textwrap

from google.cloud import bigquery
import datetime

import warnings
warnings.filterwarnings('ignore')



# function for safe queries (based on Kaggle SQL course)
def querytodf(query):
    dry_run_config = bigquery.QueryJobConfig(dry_run=True)

    # API request - dry run query to estimate costs
    dry_run_query_job = client.query(query, job_config=dry_run_config)
    print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

    # Only run the query if it's less than 1 GB
    ONE_GB = 1000*1000*1000
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_GB)

    # Set up the query (will only run if it's less than 1 GB)
    safe_query_job = client.query(query, job_config=safe_config)

    return safe_query_job.to_dataframe()

# get 'bls' dataset
client = bigquery.Client()
dataset_ref = client.dataset("bls", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)



# check column info for CES series table
table_ref = dataset_ref.table('employment_hours_earnings_series')
table = client.get_table(table_ref)



# supersector codes
url = 'https://download.bls.gov/pub/time.series/ce/ce.supersector'
dfss = pd.read_csv(url, sep='\t')
# display(table.schema[1], '', dfss)

# industry codes
url = 'https://download.bls.gov/pub/time.series/ce/ce.industry'
dfi = pd.read_csv(url, sep='\t')
# display(table.schema[2], '', dfi)

def indlvl(num):
    return dfi[dfi.display_level == num].industry_name

# datatype codes
url = 'https://download.bls.gov/pub/time.series/ce/ce.datatype'
dft = pd.read_csv(url, sep='\t')
# display(table.schema[3], '', dft)

# split long text (https://community.plotly.com/t/wrap-long-text-in-title-in-dash/11419)
def split(text, width):
    split_text = textwrap.wrap(text, width=width)
    return '<br>'.join(split_text)

In [ ]:
# display(plt.style.available)

pio.renderers.default = 'notebook_connected'
display(pio.renderers)

pio.templates.default = 'gridon'
display(pio.templates)

In [ ]:
display(plt.style.available)

plt.style.use('default')

# plt.style.use('bmh')
plt.style.use('fivethirtyeight')
# plt.style.use('seaborn-whitegrid')
# plt.style.use('seaborn-white')

plt.style.use('seaborn-notebook')

# sns.set_palette('tab20')
# sns.set_palette('Set2')

# How did COVID-19 Impact the U.S. Labor Market?
# How is the U.S. Labor Market Recovering from COVID-19?

This notebook is inspired by some of the findings here:

https://www.kaggle.com/heisxiang/are-price-levels-increasing-faster-than-wages

namely that average wages rose dramatically during the onset of Covid-19

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_colwidth', None)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from matplotlib import pyplot as plt
import seaborn as sns

from google.cloud import bigquery
import datetime

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# function for safe queries (based on Kaggle SQL course)
def querytodf(query):
    dry_run_config = bigquery.QueryJobConfig(dry_run=True)

    # API request - dry run query to estimate costs
    dry_run_query_job = client.query(query, job_config=dry_run_config)
    print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

    # Only run the query if it's less than 1 GB
    ONE_GB = 1000*1000*1000
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_GB)

    # Set up the query (will only run if it's less than 1 GB)
    safe_query_job = client.query(query, job_config=safe_config)

    return safe_query_job.to_dataframe()

# get 'bls' dataset
client = bigquery.Client()
dataset_ref = client.dataset("bls", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)

# Examine Dataset to Build Queries
Explore the data to find what we want to query

Available tables:

In [ ]:
# list of tables
tables = client.list_tables(dataset_ref)
for table in tables: print(table.table_id)

Column info and first 5 rows of `employment_hours_earnings` table:

In [ ]:
# check column info for CES table
table_ref = dataset_ref.table('employment_hours_earnings')
table = client.get_table(table_ref)
display(table.schema)

# show first 5 rows
client.list_rows(table, max_results=5).to_dataframe()

Each set of time series data has a corresponding `series_id` and `series_title`. We need to find the relevant `series_id`'s that will help answer our question. To do this we'll examine the `employment_hours_earnings_series` table which has additional info about each `series_id`.

Column info and first 5 rows of `employment_hours_earnings_series` table:

In [ ]:
# check column info for CES series table
table_ref = dataset_ref.table('employment_hours_earnings_series')
table = client.get_table(table_ref)
display(table.schema)

# show first 5 rows
client.list_rows(table, max_results=5).to_dataframe()

We can see that each `series_id` corresponds with a number of codes describing its data. We'll explore the possible values of each code and cherry pick the ones relevant to our analysis. Files from https://download.bls.gov/pub/time.series/ce/

`supersector_code`:

In [ ]:
url = 'https://download.bls.gov/pub/time.series/ce/ce.supersector'
dfss = pd.read_csv(url, sep='\t')
display(table.schema[1], '', dfss)

`industry_code`:

In [ ]:
url = 'https://download.bls.gov/pub/time.series/ce/ce.industry'
dfi = pd.read_csv(url, sep='\t')
display(table.schema[2], '', dfi)

In [ ]:
def indlvl(num):
    return dfi[dfi.display_level == num].industry_name

# for lvl in dfi.display_level.unique():
#     display('Industry level ' + str(lvl), indlvl(lvl), '')

`data_type_code`:

In [ ]:
url = 'https://download.bls.gov/pub/time.series/ce/ce.datatype'
dft = pd.read_csv(url, sep='\t')
display(table.schema[3], '', dft)

`seasonal`:

In [ ]:
url = 'https://download.bls.gov/pub/time.series/ce/ce.seasonal'
df = pd.read_csv(url, sep='\t')
display(table.schema[4], '', df)

`footnote_codes`:

In [ ]:
url = 'https://download.bls.gov/pub/time.series/ce/ce.footnote'
df = pd.read_csv(url, sep='\t')
display(table.schema[6], '', df)

There is a lot of available data, the bls.gov website provides explanations here: https://www.bls.gov/web/empsit/cestn.htm

It looks like each industry has a corresponding level. We'll keep this in mind for later in our analysis. For now we'll take all sectors, all industries, seasonally adjusted data to eliminate noise, and the following data types:

Relevant data types for this analysis:
* 1	ALL EMPLOYEES, THOUSANDS
* 2	AVERAGE WEEKLY HOURS OF ALL EMPLOYEES
* 3	AVERAGE HOURLY EARNINGS OF ALL EMPLOYEES
* 6	PRODUCTION AND NONSUPERVISORY EMPLOYEES, THOUSANDS
* 7	AVERAGE WEEKLY HOURS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES
* 8	AVERAGE HOURLY EARNINGS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES
* 11	AVERAGE WEEKLY EARNINGS OF ALL EMPLOYEES
* 30	AVERAGE WEEKLY EARNINGS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES
* 56	AGGREGATE WEEKLY HOURS OF ALL EMPLOYEES, THOUSANDS
* 57	AGGREGATE WEEKLY PAYROLLS OF ALL EMPLOYEES, THOUSANDS
* 81	AGGREGATE WEEKLY HOURS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES, THOUSANDS
* 82	AGGREGATE WEEKLY PAYROLLS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES, THOUSANDS

from `ce.text` located at https://download.bls.gov/pub/time.series/ce/:
<PRE> </PRE>
asdfasdfasdf

# Query + Prepare Data

Build query based on conditions determined above:

In [ ]:
# build query
query = """ 
SELECT 
    ces.series_id, 
    ces_s.data_type_code, 
    ces_s.supersector_code, 
    ces_s.industry_code, 
    ces.year, 
    ces.period, 
    ces.date, 
    ces.value
FROM `bigquery-public-data.bls.employment_hours_earnings` ces
INNER JOIN `bigquery-public-data.bls.employment_hours_earnings_series` ces_s
    ON ces.series_id = ces_s.series_id
WHERE ces_s.data_type_code IN (1,2,3,6,7,8,11,30,56,57,81,82)
AND ces_s.seasonal = 'S'
AND EXTRACT(YEAR FROM ces.date) >= 2006
ORDER BY ces.series_id, ces.date
"""

# query data
dfall = querytodf(query)

In [ ]:
### data cleaning + feature engineering

# replace codes with names
dfall.rename(columns = {'data_type_code': 'data_type', 
                       'supersector_code': 'supersector', 
                       'industry_code': 'industry'}, 
            inplace=True)
dfall['data_type'] = dfall['data_type'].replace(list(dft.data_type_code), 
                                                        list(dft.data_type_text))
dfall['supersector'] = dfall['supersector'].replace(list(dfss.supersector_code), 
                                                            list(dfss.supersector_name))
dfall['industry'] = dfall['industry'].replace(list(dfi.industry_code), 
                                                      list(dfi.industry_name))
dfall['ind_lvl'] = dfall['industry'].replace(list(dfi.industry_name), 
                                                      list(dfi.display_level))

# split 'data_type column' for easier slicing
dict1 = {
    'ALL EMPLOYEES, THOUSANDS': 'EMPLOYMENT OF ALL, THOUSANDS', 
    'PRODUCTION AND NONSUPERVISORY EMPLOYEES, THOUSANDS': 'EMPLOYMENT OF PRODUCTION AND NONSUPERVISORY, THOUSANDS'
}
dfall['data_type'] = dfall.data_type.replace(dict1)
newcols = dfall.data_type.str.split(', ', expand=True)
# display(newcols)
newcols[[0,2]] = newcols[0].str.split(' OF ', expand=True)
# display(newcols)
newcols[0] = (newcols[0] + ', ' + newcols[1].fillna('')).str.rstrip(', ')
newcols[2] = newcols[2].str.replace(' EMPLOYEES', '')
newcols.drop(1, axis=1, inplace=True)
dfall[['data_type', 'employee_type']] = newcols

# rename industries belonging to multiple supersectors
df=dfall.groupby(
    ['data_type', 'industry', 'employee_type']
).series_id.nunique().sort_values(ascending=False).reset_index()
i = df[df.series_id == 2].industry.unique()
display(dfall[dfall.industry.isin(i)].groupby('industry').supersector.unique())
def rename(row):
    if row.industry in i:
        row.industry = f'{row.industry} ({row.supersector})'
    return row
dfall = dfall.apply(rename, axis=1)

# reorder columns
emp = dfall['employee_type']
dfall.drop('employee_type', axis=1, inplace=True)
dfall.insert(loc=2, column='employee_type', value=emp)

lvl = dfall['ind_lvl']
dfall.drop('ind_lvl', axis=1, inplace=True)
dfall.insert(loc=5, column='ind_lvl', value=lvl)

display(dfall)

In [ ]:
# variables for slicing data

types = pd.Series(dfall.data_type.unique())
display('types:', types, '')

employees = dfall.employee_type.unique()
display('employees:', employees, '')

sectors = pd.Series(dfall.supersector.unique())
display('sectors:', sectors, '')

industries = dfall.industry.unique()
# display(industries)

# function for getting industry names by level
def indlvl(lvl):
    return dfi[dfi.display_level == lvl].industry_name

c19p = datetime.date(2020,1,1)
c19a = datetime.date(2020,2,1)
c19b = datetime.date(2020,4,1)

cdate19 = dfall.year == 2019
cdatep = dfall.date == c19p
cdatea = dfall.date == c19a
cdateb = dfall.date == c19b

In [ ]:
# create 'sup employees'

df = dfall.groupby(['data_type', 'industry']).employee_type.nunique().reset_index()
i = df[df.employee_type == 2].industry
display(i)

dftemp = dfall[
    (dfall.data_type == types[0])
    & (dfall.industry == i[0])
]

dftemp


In [ ]:
# transform queried data into final form

# final DF
# columns = ['series_id', 'data_type', 'supersector', 'industry', 'ind_lvl', 'employee', 'Avg2019', 'Jan2020', 'Feb2020', 'Apr2020', 'change', '% change']

cols = ['series_id', 
        'data_type', 
        'supersector', 
        'industry', 
        'ind_lvl', 
        'employee_type']

df0 = dfall[cdate19].groupby(cols).value.mean().rename('Avg2019').reset_index()
df0['Jan2020'] = dfall[cdatep].groupby(cols).value.mean().values
df0['Feb2020'] = dfall[cdatea].groupby(cols).value.mean().values
df0['Apr2020'] = dfall[cdateb].groupby(cols).value.mean().values
df0['change'] = df0['Apr2020'] - df0['Feb2020']
df0['% change'] = (df0['Apr2020'] - df0['Feb2020']) / df0['Feb2020']

display(df0)

In [ ]:
dftemp = data[
    dfall.date.isin([c19p, cdatea, cdateb])
]

dftemp

# EDA

time-series simple viz: e,h,e (all vs ns)

In [ ]:
dfall.isna().sum()

In [ ]:
dfall.info()

In [ ]:
# categorical variables
cat = dfall.columns[[0,1,2,3,4,6]]
cat

In [ ]:
for c in cat:
    display(c, dfall[c].nunique(), '')

In [ ]:
display(
    'min dates:', dfall.groupby('series_id').date.min().value_counts(), '',
    'max dates:', dfall.groupby('series_id').date.max().value_counts(), '', 
    'series per data_type:', dfall.groupby('data_type').series_id.nunique().sort_values(), '', 
    'series per supersector:', dfall.groupby('supersector').series_id.nunique().sort_values(), '',
    'series per industry:', dfall.groupby('industry').series_id.nunique().sort_values(), '',
)

In [ ]:
# industries per supersector
dfall.groupby('supersector').industry.value_counts()

In [ ]:
# check for missing months
s = dfall.groupby('series_id').period.value_counts().rename('period_counts').to_frame().reset_index()
display(s)
s2 = s.groupby('series_id').period_counts.max() - s.groupby('series_id').period_counts.min()
s2.value_counts() # max should be 1

# Pre-COVID

time-series simple viz: e,h,e (all vs ns)

pre-covid:
* sup vs ns employees (ratio): emp, hours, earnings
* avg hours/earnings

Industry size: emp, hours, earnings
* avg hours/earnings
* which ind has highest sup-ns ratio?
* which ind has largest wage gap between sup-ns? hours gap?

pre-covid trends:
* overall growth rate
* by industry: highest/lowest growth

time-series simple viz: e,h,e (all vs ns)

In [ ]:
# basic data types
ctypes = dfall.data_type.isin(types[[0,4,5]])
cdates = dfall.date >= datetime.date(2016,1,1)
cind = dfall.industry == 'Total private'

df = dfall[ctypes&cdates&cind]

# g = sns.relplot(data=df, x='date', y='value', kind='line', row='data_type', 
#             hue='employee_type', 
#             facet_kws=dict(sharey=False), 
#             height=2.3, aspect=3)
# g.fig.subplots_adjust(top=0.85)
# g.fig.suptitle('Basic Data Types (employment, hours, earnings)');

fig = px.line(df, x='date', y='value', color='employee_type', facet_col='data_type', facet_col_wrap=1, title='Basic Data Types')
fig.update_yaxes(matches=None)
fig.update_layout(width=720, height=640)

# fig.add_vrect(x0='2020-02-01', x1='2020-04-01',
#               annotation_text='', annotation_position="top left",
#               fillcolor='red', opacity=0.15, line_width=0)

fig.show()

In [ ]:
# derived data types
ctypes = data.data_type.isin(types[[1,2,3]])
cdates = data.date >= datetime.date(2016,1,1)
cind = data.industry == 'Total private'

df = data[ctypes&cdates&cind]

fig = px.line(df, x='date', y='value', color='employee_type', facet_col='data_type', facet_col_wrap=1, title='Derived Data Types')
fig.update_yaxes(matches=None)
fig.update_layout(width=720, height=640)
fig.show()

### Largest/Smallest industries? (pre-COVID)

by value on Jan 2020:

In [ ]:
basic = types[[0,4,5]]

indsize = df0
indsize = indsize[indsize.data_type.isin(basic)
                 & (indsize.employee_type == 'ALL')]

indsize = indsize[cols + ['Jan2020']]
lvls = np.sort(indsize.ind_lvl.unique())
# display(indsize)

dftemp = pd.DataFrame()
for typ in basic:
    for lvl in lvls:
        dftemp = dftemp.append(
            indsize[(indsize.data_type == typ)
                    & (indsize.ind_lvl == lvl)
                   ].sort_values(by='Jan2020').tail(10)
        )
display(dftemp)
# for typ in basic:
#     for lvl in lvls:
#         dftemp = indsize[(indsize.data_type == typ)
#                          & (indsize.ind_lvl == lvl)
#                         ].groupby('industry').Jan2020.sum().sort_values().reset_index()
#         fig = px.bar(dftemp.tail(20), x='Jan2020', y='industry', 
#                      title=typ+' level '+str(lvl)+', '+str(indsize[indsize.ind_lvl == lvl].industry.nunique())+' total industries', )
#         fig.update_yaxes(automargin=True, matches=None)
#         fig.update_layout(width=720, height=dftemp.tail(20).industry.nunique()*20+200)
#         fig.for_each_trace(lambda trace: trace.update(name=split(trace.name, 31)))
#         fig.show()

fig = px.bar(dftemp, y='industry', x='Jan2020',
             facet_col = 'data_type',
             facet_row = 'ind_lvl'
            )
fig.update_yaxes(showticklabels=True, automargin=True, matches=None)
fig.update_xaxes(matches=None)
fig.update_layout(height=1800)
fig.show()

In [ ]:
dftemp = indsize[
    (indsize.data_type == basic[0])
    & (indsize.ind_lvl == 3)
].groupby('industry').Jan2020.sum().sort_values().reset_index()
dftemp

fig = px.bar(dftemp.tail(10), y='industry', x='Jan2020')
fig.update_yaxes(automargin=True, 
                 tickvals=dftemp.tail(10).reset_index().industry.index, 
                 ticktext=[split(name, 25) for name in dftemp.tail(10).reset_index().industry.values],
                 ticklabelposition='outside'
                )
fig.update_layout(height=480, width=640)
fig.show()

In [ ]:
[split(name, 30) for name in dftemp.industry.values]

In [ ]:
# eda for wage levels pre-covid

AWE = types[3]
display(AWE)
ind = indlvl(3)
display(ind)

df = data[(data.data_type == AWE)
          & (data.industry.isin(ind))
         ]
# display(df)

# this should always be 1:
df.groupby(['supersector','industry', 'employee']).series_id.nunique().value_counts()

In [ ]:
# industries by pre-covid wages


AWE = types[3]
display(AWE)
ind = indlvl(3)
display(ind)

df = data[(data.data_type == AWE)
          & (data.industry.isin(ind))
          & (data.date == c19p)]

display(df.groupby(['supersector','industry', 'employee']).value.mean().sort_values(ascending=False).reset_index())

# How did COVID-19 Impact the U.S. Labor Market?

### Outline
Definitions from https://www.bls.gov/web/empsit/cestn.htm#section5:

Basic data types:
* Total employees
* Total weekly payroll
* Total weekly hours

Derived data types:
* Average weekly hours = total weekly hours / total employees
* Average hourly earnings = total weekly payroll / total weekly hours
* Average weekly earnings = average hourly earnings * average weekly hours (total weekly payroll / total employees)

Hypothesis:
* Many of the people who lost their jobs during COVID-19 were relatively lower wage earners, thus leaving relatively higher wage earners in the payroll data, driving average wages higher.

Method: 

Compare pre vs during COVID-19 numbers for each of the datatypes

**Total private sector**
* AHE vs AWE
* Basic Data
    * All employees vs non-supervisory
    * abs vs pct
* AWH: total hours vs total employees
* AHE: payroll vs total hours
* AWE: AHE vs AWH
* AWE: payroll vs total employees

**by industry:**
* Basic data: change by industry (tiered by average earnings pre-covid)
* Hours change?

### Total Private Sector

* Basic Data
    * All employees vs non-supervisory
    * abs vs pct

In [ ]:
ind = indlvl(1).values[0]
typ = types[4]
c19a = datetime.date(2020,2,1)
c19b = datetime.date(2020,4,1)

cdate1 = data.date >= datetime.date(2019,7,1)
cdate2 = data.date <= datetime.date(2020,12,1)
ctype = data.data_type == typ
cind = data.industry == ind

dfog = data[cdate1&cdate2&ctype&cind]
df = dfog.groupby(['data_type', 'employee', 'date']).value.sum().reset_index()

# difference between start and worst of drop
df0 = df[df.employee == df.employee.unique()[0]]
start0 = df0[df0.date == c19a].value.mean()
end0 = df0[df0.date == c19b].value.mean()
diff0 = end0 - start0

df1 = df[df.employee == df.employee.unique()[1]]
start1 = df1[df1.date == c19a].value.mean()
end1 = df1[df1.date == c19b].value.mean()
diff1 = end1 - start1

plt.figure(figsize=(6,3))
sns.lineplot(data=df, x='date', y='value', hue='employee',)
plt.title('COVID-19 Impact: U.S. ' + ind + ', ' + typ, y=1.3)
plt.legend(loc='lower left', bbox_to_anchor=(-0.2,1.05))

plt.axvspan(datetime.date(2020,2,1), datetime.date(2020,4,1), 
            color='red', alpha=0.2)

plt.text(datetime.date(2020,4,1), start0+diff0/2, 
         str('{:,}'.format(round(diff0)))+'\n({:.2%})'.format(diff0/start0),
         color = sns.color_palette()[0],
         weight='bold')
plt.text(datetime.date(2020,4,1), start1+diff1/2, 
         str('{:,}'.format(round(diff1)))+'\n({:.2%})'.format(diff1/start1),
         color = sns.color_palette()[1],
         weight='bold')

plt.xticks(rotation=20); 

Average weekly hours = total weekly hours / total employees

AWH: total hours vs total employees

### by industry

In [ ]:
indlvl(3)

In [ ]:
df2 = data[ctype
           & data.industry.isin(indlvl(4).values)
           & data.date.isin([c19a, c19b])]
df2 = df2.groupby(['industry', 'data_type', 'employee', 'date']).value.sum().reset_index()
df2 = df2.pivot(index=['industry', 'data_type', 'employee'], columns='date', values='value').reset_index()
df2.columns.name = None
df2['diff'] = df2[df2.columns[4]] - df2[df2.columns[3]]
df2['% change'] = (df2[df2.columns[4]] - df2[df2.columns[3]])/df2[df2.columns[3]]
df2

In [ ]:
order = df2[df2.employee == 'ALL EMPLOYEES'].groupby('industry')['% change'].min().sort_values().head(15)
plt.figure(figsize=(4,6))
sns.barplot(data=df2, y='industry', x='% change', hue='employee', order=order.index)
plt.legend(loc='upper left', bbox_to_anchor=(-0.5,-0.1))

order = df2[df2.employee == 'ALL EMPLOYEES'].groupby('industry')['% change'].min().sort_values().tail(15)
plt.figure(figsize=(4,6))
sns.barplot(data=df2, y='industry', x='% change', hue='employee', order=order.index)
plt.legend(loc='upper left', bbox_to_anchor=(-0.5,-0.1))

In [ ]:
order = df2.groupby('industry')[df2.columns[3]].max().sort_values(ascending=False).head(20)
plt.figure(figsize=(4,6))
sns.barplot(data=df2, y='industry', x=df2.columns[3], hue='employee', order=order.index)
plt.legend(loc='upper left', bbox_to_anchor=(-0.5,-0.1))

In [ ]:
# sns.set_palette('Dark2')
sns.color_palette('tab10_r')
# sns.color_palette()

# WIP

In [ ]:
url = 'https://download.bls.gov/pub/time.series/le/le.series'
df = pd.read_csv(url, sep='\t')

url = 'https://download.bls.gov/pub/time.series/le/le.data.0.Current'
df1 = pd.read_csv(url, sep='\t')

In [ ]:
# earn_code 01
# fips_code 00
# indy_code 0000

In [ ]:
cond1 = df.earn_code == 1
cond2 = df.fips_code == 0
cond3 = df.indy_code == 0

df[cond1 & cond2 & cond3].iloc[:,0:4]

In [ ]:
a = df1.columns[0]
v = df1.columns[3]

display(a, v)

In [ ]:
data = df1[df1[a] == df1[a][0]]
data[v] = data[v].str.lstrip(' ').astype(float)

plt.figure(figsize=(8,4))
sns.lineplot(data = data[data.year > 2016], x=data[data.year > 2016].index, y=v);

In [ ]:
data[data.year >= 2019]